In [2]:
# Import dependencies
import pandas as pd 
import sqlite3

from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

# Removing engine init for now 
#engine = create_engine("sqlite:///birthdata.sqlite", echo=False)

## Cleaning County-Based Clinic Data

In [3]:
# Import csv file 
clinic_file = "Guttmacher_Clinic_Census_County_2015.csv"
clinic_df = pd.read_csv(clinic_file)
clinic_df 

,State,FIPS,County,Total publicly funded clinics,Number of federally qualified health centers,Number of health department clinics,Number of hospital-based clinics,Number of Planned Parenthood clinics,Number of other clinics,Total Title X-funded clinics,...,Number of female contraceptive clients served at health department clinics,Number of female contraceptive clients served at hospital-based clinics,Number of female contraceptive clients served at Planned Parenthood clinics,Number of female contraceptive clients served at other clinics,Total number of female contraceptive clients served at Title X-funded clinics,Number of female contraceptive clients served at Title X-funded federally qualified health centers,Number of female contraceptive clients served at Title X-funded health department clinics,Number of female contraceptive clients served at Title X-funded hospital-based clinics,Number of female contraceptive clients served at Title X-funded Planned Parenthood clinics,Number of female contraceptive clients served at Title X-funded other clinics
0,US Total,NaN,NaN,10708.0,5829.0,2242.0,853.0,676.0,1108.0,3700.0,...,1251190.0,612290.0,1996940.0,510160.0,3827650,711550.0,1083120.0,196610.0,1581760.0,254620.0
1,Alabama,NaN,NaN,187.0,106.0,77.0,1.0,2.0,1.0,83.0,...,79510.0,1250.0,260.0,60.0,86180,7280.0,78900.0,0.0,0.0,0.0
2,Alaska,NaN,NaN,157.0,100.0,16.0,0.0,5.0,36.0,5.0,...,3560.0,0.0,3670.0,3540.0,5290,0.0,1290.0,0.0,3360.0,630.0
3,Arizona,NaN,NaN,232.0,125.0,33.0,18.0,12.0,44.0,36.0,...,11110.0,4030.0,23930.0,14050.0,31820,3740.0,8970.0,650.0,16750.0,1700.0
4,Arkansas,NaN,NaN,163.0,69.0,92.0,0.0,2.0,0.0,92.0,...,47450.0,0.0,4590.0,0.0,51510,0.0,46920.0,0.0,4590.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3190,Wyoming,56039.0,Teton,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
3191,Wyoming,56041.0,Uinta,1.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,190.0,190,0.0,0.0,0.0,0.0,190.0
3192,Wyoming,56043.0,Washakie,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,210.0,0.0,0.0,0.0,210,0.0,210.0,0.0,0.0,0.0
3193,Wyoming,56045.0,Weston,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,160.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0


In [12]:
# Renaming columns 
#clinic_df.rename(columns={'A': 'a', 'B':'b'}, inplace=True)

# Drop unnecessary columns
#clinic_df = clinic_df[['a', 'b']]

clinic_df.head(55) # Note that 0 to 51 are statewide, NOT county

,State,FIPS,County,Total publicly funded clinics,Number of federally qualified health centers,Number of health department clinics,Number of hospital-based clinics,Number of Planned Parenthood clinics,Number of other clinics,Total Title X-funded clinics,...,Number of female contraceptive clients served at health department clinics,Number of female contraceptive clients served at hospital-based clinics,Number of female contraceptive clients served at Planned Parenthood clinics,Number of female contraceptive clients served at other clinics,Total number of female contraceptive clients served at Title X-funded clinics,Number of female contraceptive clients served at Title X-funded federally qualified health centers,Number of female contraceptive clients served at Title X-funded health department clinics,Number of female contraceptive clients served at Title X-funded hospital-based clinics,Number of female contraceptive clients served at Title X-funded Planned Parenthood clinics,Number of female contraceptive clients served at Title X-funded other clinics
0,US Total,NaN,NaN,10708.0,5829.0,2242.0,853.0,676.0,1108.0,3700.0,...,1251190.0,612290.0,1996940.0,510160.0,3827650,711550.0,1083120.0,196610.0,1581760.0,254620.0
1,Alabama,NaN,NaN,187.0,106.0,77.0,1.0,2.0,1.0,83.0,...,79510.0,1250.0,260.0,60.0,86180,7280.0,78900.0,0.0,0.0,0.0
2,Alaska,NaN,NaN,157.0,100.0,16.0,0.0,5.0,36.0,5.0,...,3560.0,0.0,3670.0,3540.0,5290,0.0,1290.0,0.0,3360.0,630.0
3,Arizona,NaN,NaN,232.0,125.0,33.0,18.0,12.0,44.0,36.0,...,11110.0,4030.0,23930.0,14050.0,31820,3740.0,8970.0,650.0,16750.0,1700.0
4,Arkansas,NaN,NaN,163.0,69.0,92.0,0.0,2.0,0.0,92.0,...,47450.0,0.0,4590.0,0.0,51510,0.0,46920.0,0.0,4590.0,0.0
5,California,NaN,NaN,1697.0,833.0,94.0,371.0,116.0,283.0,353.0,...,60300.0,160040.0,712470.0,137370.0,1014320,242160.0,36930.0,3050.0,704630.0,27550.0
6,Colorado,NaN,NaN,221.0,139.0,43.0,10.0,22.0,7.0,64.0,...,26840.0,8360.0,40930.0,6030.0,50280,12970.0,25530.0,6010.0,0.0,5780.0
7,Connecticut,NaN,NaN,92.0,61.0,0.0,9.0,17.0,5.0,20.0,...,0.0,5130.0,54720.0,630.0,46790,5080.0,0.0,50.0,41330.0,330.0
8,Delaware,NaN,NaN,41.0,9.0,22.0,0.0,3.0,7.0,38.0,...,5140.0,0.0,4200.0,1680.0,13480,3130.0,4670.0,0.0,4200.0,1480.0
9,District of Columbia,NaN,NaN,31.0,27.0,0.0,3.0,1.0,0.0,23.0,...,0.0,2050.0,2900.0,0.0,30750,30750.0,0.0,0.0,0.0,0.0


In [5]:
# Check data types
clinic_df.dtypes

State                                                                                                  object
FIPS                                                                                                  float64
County                                                                                                 object
Total publicly funded clinics                                                                         float64
Number of federally qualified health centers                                                          float64
Number of health department clinics                                                                   float64
Number of hospital-based clinics                                                                      float64
Number of Planned Parenthood clinics                                                                  float64
Number of other clinics                                                                               float64
Total Titl

## Cleaning County-Based Need Data

In [6]:
# Import csv file 
need_file = "Guttmacher_Need_County_2016.csv"
need_df = pd.read_csv(need_file)
need_df

,measure_name,datum,county_id,state_id,state_name,first_year,last_year,footnotes,sources
0,No. of women who likely need public support fo...,3650,1001,AL,Alabama,2016.0,NaN,NaN,Publicly Supported Family Planning Services in...
1,No. of women who likely need public support fo...,11500,1003,AL,Alabama,2016.0,NaN,NaN,Publicly Supported Family Planning Services in...
2,No. of women who likely need public support fo...,1640,1005,AL,Alabama,2016.0,NaN,NaN,Publicly Supported Family Planning Services in...
3,No. of women who likely need public support fo...,1480,1007,AL,Alabama,2016.0,NaN,NaN,Publicly Supported Family Planning Services in...
4,No. of women who likely need public support fo...,3490,1009,AL,Alabama,2016.0,NaN,NaN,Publicly Supported Family Planning Services in...
...,...,...,...,...,...,...,...,...,...
675,Total no. of women aged 20-44 with potential d...,6050,1127,AL,Alabama,2016.0,NaN,NaN,Publicly Supported Family Planning Services in...
676,Total no. of women aged 20-44 with potential d...,1460,1129,AL,Alabama,2016.0,NaN,NaN,Publicly Supported Family Planning Services in...
677,Total no. of women aged 20-44 with potential d...,1000,1131,AL,Alabama,2016.0,NaN,NaN,Publicly Supported Family Planning Services in...
678,Total no. of women aged 20-44 with potential d...,1860,1133,AL,Alabama,2016.0,NaN,NaN,Publicly Supported Family Planning Services in...


In [14]:
# Reorient the table: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.transpose.html
# Use transpose to rotate the table?

# Renaming columns 
#need_df.rename(columns={'A': 'a', 'B':'b'}, inplace=True)

# Drop unnecessary columns
#need_df = need_df[['a', 'b']]
need_df.head(10)


,measure_name,datum,county_id,state_id,state_name,first_year,last_year,footnotes,sources
0,No. of women who likely need public support fo...,3650,1001,AL,Alabama,2016.0,NaN,NaN,Publicly Supported Family Planning Services in...
1,No. of women who likely need public support fo...,11500,1003,AL,Alabama,2016.0,NaN,NaN,Publicly Supported Family Planning Services in...
2,No. of women who likely need public support fo...,1640,1005,AL,Alabama,2016.0,NaN,NaN,Publicly Supported Family Planning Services in...
3,No. of women who likely need public support fo...,1480,1007,AL,Alabama,2016.0,NaN,NaN,Publicly Supported Family Planning Services in...
4,No. of women who likely need public support fo...,3490,1009,AL,Alabama,2016.0,NaN,NaN,Publicly Supported Family Planning Services in...
5,No. of women who likely need public support fo...,710,1011,AL,Alabama,2016.0,NaN,NaN,Publicly Supported Family Planning Services in...
6,No. of women who likely need public support fo...,1550,1013,AL,Alabama,2016.0,NaN,NaN,Publicly Supported Family Planning Services in...
7,No. of women who likely need public support fo...,8430,1015,AL,Alabama,2016.0,NaN,NaN,Publicly Supported Family Planning Services in...
8,No. of women who likely need public support fo...,2560,1017,AL,Alabama,2016.0,NaN,NaN,Publicly Supported Family Planning Services in...
9,No. of women who likely need public support fo...,1390,1019,AL,Alabama,2016.0,NaN,NaN,Publicly Supported Family Planning Services in...


In [11]:
# Check data types
need_df.dtypes

measure_name     object
datum             int64
county_id        object
state_id         object
state_name       object
first_year      float64
last_year       float64
footnotes        object
sources          object
dtype: object

## Cleaning Outcomes Data

In [8]:
# Are we interested in using this data? 
# Import csv file 
outcomes_file = "Guttmacher_Outcomes_2016.csv"
outcomes_df = pd.read_csv(outcomes_file)
outcomes_df

,U.S. State,"No. of pregnancies averted by Title X-funded centers, 2016","No. of births averted by Title X-funded centers, 2016","No. of abortions averted by Title X-funded centers, 2016","No. of pregnancies averted to clients aged <20 by Title X-funded centers, 2016","No. of Title X-funded centers, 2015","No. of female contraceptive clients served at Title X-funded centers, 2016","No. of female contraceptive clients served at publicly funded centers , 2016","No. of female contraceptive clients younger than 20 served at publicly funded centers, 2016","No. of female contraceptive clients younger than 20 served at Title X-funded centers, 2016","No. of publicly funded family planning centers, 2015","No. of pregnancies averted by publicly funded centers, 2016","No. of births averted by publicly funded centers, 2016","No. of abortions averted by publicly funded centers, 2016","No. of pregnancies averted to clients aged <20 by publicly funded centers, 2016","% of likely need met by Title X-funded centers, 2016","% of likely need met by publicly funded centers, 2016"
0,U.S. total,"755,300","355,650","255,550","172,490","3,700","3,522,120","6,088,190","1,166,960","626,970","10,708","1,305,570","614,760","441,730","321,050",17,18
1,Alabama,"18,330","8,630","6,200","4,680",83,"85,460","97,600","20,300","17,010",187,"20,930","9,850","7,080","5,580",24,28
2,Alaska,"1,560",740,530,390,5,"7,280","21,640","4,750","1,400",157,"4,640","2,190","1,570","1,310",18,54
3,Arizona,"6,570","3,090","2,220","1,610",36,"30,620","88,140","16,500","5,840",232,"18,900","8,900","6,390","4,540",7,19
4,Arkansas,"10,040","4,730","3,400","2,590",92,"46,840","50,960","10,610","9,400",163,"10,930","5,150","3,700","2,920",21,23
5,California,"194,280","91,480","65,730","38,590",353,"905,950","1,618,010","252,840","140,280","1,697","346,970","163,380","117,390","69,560",36,64
6,Colorado,"8,940","4,210","3,020","2,630",64,"41,680","115,490","23,720","9,570",221,"24,770","11,660","8,380","6,530",12,35
7,Connecticut,"7,420","3,500","2,510","1,550",20,"34,620","68,130","14,160","5,630",92,"14,610","6,880","4,940","3,900",19,38
8,Delaware,"3,220","1,510","1,090","1,150",38,"15,000","16,080","4,510","4,180",41,"3,450","1,620","1,170","1,240",28,30
9,District of Columbia,"8,070","3,800","2,730","1,750",23,"37,650","43,600","7,370","6,350",31,"9,350","4,400","3,160","2,030",76,88


In [ ]:
# Renaming columns 
#outcomes_df.rename(columns={'A': 'a', 'B':'b'}, inplace=True)

# Drop unnecessary columns
#outcomes_df = outcomes_df[['a', 'b']]
outcomes_df.head(10)


In [15]:
outcomes_df.dtypes

U.S. State                                                                                     object
No. of pregnancies averted by Title X-funded centers, 2016                                     object
No. of births averted by Title X-funded centers, 2016                                          object
No. of abortions averted by Title X-funded centers, 2016                                       object
No. of pregnancies averted to clients aged <20 by Title X-funded centers, 2016                 object
No. of Title X-funded centers, 2015                                                            object
No. of female contraceptive clients served at Title X-funded centers, 2016                     object
No. of female contraceptive clients served at publicly funded centers , 2016                   object
No. of female contraceptive clients younger than 20 served at publicly funded centers, 2016    object
No. of female contraceptive clients younger than 20 served at Title X-funded cente

## Exporting to SQLite

In [53]:
# Export dfs to SQLite
#clinic_df.to_sql(name='clinic', con=engine)
#need_df.to_sql(name='need', con=engine)
#add any additional ones 


### Double-Checking Classes and Work

In [2]:
# Checking classes were made successfully: https://stackoverflow.com/questions/42946174/sqlalchemy-automap-not-generating-base-classes-table-name
engine = create_engine("sqlite:///birthdata.sqlite", echo=False)

In [3]:
# Declare a Base using `automap_base()`
Base = automap_base()

In [4]:
# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)

In [5]:
# Print all of the classes mapped to the Base
Base.classes.keys()

['county', 'national']

In [6]:
# Assign the classes to variables
Clinic = Base.classes.clinic
Need = Base.classes.need

In [7]:
# Create a session
session = Session(engine)

In [10]:
# Update once ready 
# Clinic: Display the row's columns and data in dictionary format
#first_row_national = session.query(National).first()
#first_row_national.__dict__

In [9]:
# Update once ready 
# Need: Display the row's columns and data in dictionary format 
#first_row_county = session.query(County).first()
#first_row_county.__dict__

In [87]:
session.close()
